In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import datetime, date, time, timedelta

def label_var(data,variables_cat):
    lb=[]
    for m in variables_cat:
        l=LabelEncoder()
        lb.append(l.fit(list(data[m].dropna())))
    
    return lb

def label_enc(data,l,categorical_features):
    i=0
    for m in categorical_features:
        data.loc[data[m].notnull(),m]=l[i].transform(data.loc[data[m].notnull(),m])
        i=i+1

In [ ]:

df_tr = pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv")
df_ts = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [ ]:
df_tr.head(3)

In [ ]:
#df_tr.drop(df_tr.loc[df_tr['hospital_admit_source']=='Observation'].index, inplace=True)
#df_tr = df_tr[df_tr['hospital_admit_source']!='Observation']
df_tr['gcs'] = df_tr[['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']].mean(axis =1)
df_ts['gcs'] = df_ts[['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']].mean(axis =1)
df_tr['gcs'].fillna(df_tr['gcs'].mean(), inplace = True)
df_ts['gcs'].fillna(df_ts['gcs'].mean(), inplace = True)
df_tr.drop(['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache',
           'h1_inr_max', 'h1_inr_min', 'paco2_for_ph_apache','readmission_status',
    'd1_bilirubin_max','d1_creatinine_max','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min','d1_mbp_noninvasive_min',
    'd1_sysbp_noninvasive_max','d1_sysbp_noninvasive_min','h1_albumin_min','h1_bilirubin_min','h1_bun_min',
    'h1_creatinine_min','h1_hco3_min','h1_mbp_noninvasive_min','h1_platelets_min','h1_sysbp_noninvasive_max','h1_wbc_min','hospital_id'], axis=1)
df_ts.drop(['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache',
           'h1_inr_max', 'h1_inr_min', 'paco2_for_ph_apache','readmission_status',
    'd1_bilirubin_max','d1_creatinine_max','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min','d1_mbp_noninvasive_min',
    'd1_sysbp_noninvasive_max','d1_sysbp_noninvasive_min','h1_albumin_min','h1_bilirubin_min','h1_bun_min',
    'h1_creatinine_min','h1_hco3_min','h1_mbp_noninvasive_min','h1_platelets_min','h1_sysbp_noninvasive_max','h1_wbc_min','hospital_id'], axis=1)


df_tr['apache_2_bodysystem'] = df_tr['apache_2_bodysystem'].fillna("Undefined diagnoses")
df_ts['apache_2_bodysystem'] = df_ts['apache_2_bodysystem'].fillna("Undefined diagnoses")

eth = 'Other/Unknown'
#eth = 'Caucasian'
df_tr['ethnicity'] = df_tr['ethnicity'].fillna(eth)
df_ts['ethnicity'] = df_ts['ethnicity'].fillna(eth)

df_tr['hospital_admit_source'] = df_tr['hospital_admit_source'].fillna("Other")
df_ts['hospital_admit_source'] = df_ts['hospital_admit_source'].fillna("Other")

df_tr['icu_admit_source'] = df_tr['icu_admit_source'].fillna("Other ICU")
df_ts['icu_admit_source'] = df_ts['icu_admit_source'].fillna("Other ICU")

'''
df_tr['gender'] = df_tr['gender'].fillna("NA")
df_ts['gender'] = df_ts['gender'].fillna("NA")

categorical_features = []
for m in train_columns:
    if(df_tr[m].dtypes=='object'):
        categorical_features.append(m)

df_tr[categorical_features] = df_tr[categorical_features].fillna("NA")
df_ts[categorical_features] = df_ts[categorical_features].fillna("NA")
'''

In [ ]:

train_columns = [x for x in df_tr.columns if x not in ['encounter_id','patient_id','hospital_death','readmission_status']]

categorical_features = []
for m in train_columns:
    if(df_tr[m].dtypes=='object'):
        categorical_features.append(m)
        
df_tr_ts = pd.concat([df_tr[categorical_features],df_ts[categorical_features]])

l = label_var(df_tr_ts, categorical_features)
label_enc(df_tr,l,categorical_features)
label_enc(df_ts,l,categorical_features)

for df in [df_tr, df_ts]:
    for m in categorical_features:
        df[m] = df[m].astype(float)
        
categorical_index = [train_columns.index(x) for x in categorical_features]

target = df_tr['hospital_death']

In [ ]:
param = {'task': 'train',
         'boosting': 'gbdt',
         'objective':'binary',
         'metric': 'auc',
         'num_leaves': 15,
         'min_data_in_leaf': 90,
         'learning_rate': 0.01,
         'max_depth': 5,
         'feature_fraction': 0.1,
         'bagging_freq': 1,
         'bagging_fraction': 0.75,
         'use_missing': True,
         'nthread': 4
        }

In [ ]:

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=256)
oof = np.zeros(len(df_tr))
r=[]
predictions = np.zeros(len(df_ts))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_tr,target.values)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    
    trn_data = lgb.Dataset(df_tr.iloc[trn_idx][train_columns], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(df_tr.iloc[val_idx][train_columns], label=target.iloc[val_idx],reference=trn_data)

    num_round = 7000
    clf = lgb.train(param,trn_data,num_round,valid_sets=val_data,early_stopping_rounds=105,verbose_eval=200,categorical_feature=categorical_index)
    oof[val_idx] = clf.predict(df_tr.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
   
    a=roc_auc_score(target.loc[val_idx],clf.predict(df_tr.loc[val_idx,train_columns].values, num_iteration=clf.best_iteration))
    r.append(a)
    
    #predictions
    predictions += clf.predict(df_ts[train_columns], num_iteration=clf.best_iteration) / folds.n_splits
    
strAUC = roc_auc_score(target, oof)
print(strAUC)
print ("mean: "+str(np.mean(np.array(r))))
print ("std: "+str(np.std(np.array(r))))

df_sub = pd.DataFrame({'encounter_id': df_ts['encounter_id']})
df_sub['hospital_death'] = predictions

df_sub.to_csv("2409.csv",index=False)

In [ ]:
print('completed')